# BSBlab Python Demo

## SRU - Schnittstellenabfrage und Erstellung einer Liste mit Titeln und Links zu BSBDISCOVER! und den MDZ-Digitalisaten

Mit der Programmiersprache Python können Sie automatisiert Anfragen an die [BSB-SRU](https://www.bsb-muenchen.de/bsblab/datenschnittstellen/bsb-sru/) Schnittstelle stellen und diese auswerten. Diese Demo zeigt Ihnen an einem Beispiel, wie.
Die Demo wurde von den SRU-Tutorials des [Lab-Teams der Deutschen Nationalbibliothek](https://www.dnb.de/DE/Professionell/Services/WissenschaftundForschung/DNBLab/dnblabTutorials.html) inspiriert:
<https://www.dnb.de/DE/Professionell/Services/WissenschaftundForschung/DNBLab/dnblabTutorials.html?nn=849628#doc1038762bodyText2>

### Was tut dieses Skript?

- Eine Suchanfrage an die SRU-Schnittstelle stellen und Metadaten in MARCXML zurückgeben
- Informationen aus Metadaten extrahieren, OPAC- und MDZ-URLs für jeden Treffer konstruieren
- Ergebnisse als CSV-Datei speichern.

## Einrichten der Arbeitsumgebung <a class="anchor" id="Teil1"></a>

Für dieses Skript werden zuerst einige Python-Bibliotheken geladen:


In [ ]:
# Disable TLS-Warnings, Use only in Demo-Environment!!!!!
import urllib3

urllib3.disable_warnings()

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd


### Anfragen formulieren

Für die Abfrage der SRU-Schnittstelle wird eine URL konstruiert, hierfür nutzen wir Requests: https://docs.python-requests.org/en/latest/

### XML-Daten verarbeiten

- BeautifulSoup <https://pypi.org/project/beautifulsoup4/> wird für die Suche in den Metadaten verwendet
- [etree](https://docs.python.org/3/library/xml.etree.elementtree.html) und [unicodedata](https://docs.python.org/3/library/unicodedata.html) werden bei der Formulierung der Suchparameter und der Ergebnisauswertung genutzt.

### Ausgabe der Daten

Mit Pandas https://pandas.pydata.org/ werden die gesammelten Ergebnisse dargestellt/abgespeichert

## SRU-Abfrage mit Ausgabe in MARCXML<a class="anchor" id="Teil2"></a>

Die Funktion bsb_sru erzeugt eine URL zur Abfrage der SRU-Schnittstelle. Die eigentliche Suchanfrage entsteht im Parameter "query" und wird später formuliert. Aus dem Abfrageergebnis werden die einzelnen Titeldatensätze (recordData) extrahiert und als "records" bzw. "new_records" zurückgegeben. Bei mehr als 50 Treffern werden weitere Datensätze mit "&startRecord=" abgerufen (beginnend mit 51, die Schnittstelle liefert maximal 10.000 Treffer zurück). 

In [ ]:
def bsb_sru(query):
    base_url = "https://bsb.alma.exlibrisgroup.com/view/sru/49BVB_BSB"
    params = {
        'recordSchema': 'marcxml',
        'operation': 'searchRetrieve',
        'version': '1.2',
        'maximumRecords': '50',
        'query': query
    }

    r = requests.get(base_url, params=params)
    xml = soup(r.content, features='xml')  
    records = xml.find_all('recordData')

    if len(records) < 50:
        return records

    num_results = len(records)
    i = 51

    while num_results == 50:
        params.update({'startRecord': i})
        r = requests.get(base_url, params=params)
        xml = soup(r.content, features='xml')  
        new_records = xml.find_all('recordData')
        records += new_records
        i += 50
        num_results = len(new_records)

    return records


## Metadaten auslesen und verarbeiten<a class="anchor" id="Teil3"></a>

Die Funktion parse_record nimmt jeweils einen Titeldatensatz als "record" entgegen und filtert mit "xml.find" die gewünschten Informationen heraus. In diesem Beispiel extrahieren wir den Titel (titel), die Datensatz-ID (bsbid) des Bibliothekssystems und die ID des Digitalisats (mdzid). Die auszulesenden Informationen können hier beliebig agepasst und erweitert werden. Zuletzt werden mit den IDs jeweils eine "opac_url" und eine "mdz_url" als Links konstruiert und alle Informationen als Dictionary zurückgegeben.

In [ ]:
def parse_record(record_data):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    record = record_data.find('record')
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))

    # Extracting BSB-ID
    bsbid_elem = xml.find(".//marc:controlfield[@tag = '001']", namespaces=ns)
    bsbid = bsbid_elem.text if bsbid_elem is not None else 'fail'

    # Extracting Titel
    titel_elem = xml.find(".//marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    titel = titel_elem.text if titel_elem is not None else "unknown"

    # Extracting MDZ-ID
    mdzid_elem = xml.find(".//marc:datafield[@tag = '990']/marc:subfield[@code = 'c']", namespaces=ns)
    mdzid = mdzid_elem.text if mdzid_elem is not None else 'fail'

    # Constructing URLs
    opac_url = f"https://opacplus.bsb-muenchen.de/title/{bsbid}"
    mdz_url = f"https://mdz-nbn-resolving.de/details:{mdzid}"

    # Write to Dictionary
    meta_dict = {"Titel": titel, "BSB-ID": bsbid, "MDZ-ID": mdzid, "OPAC-URL": opac_url, "MDZ-URL": mdz_url}
    return meta_dict

## Suchanfrage formulieren

In [ ]:
records = bsb_sru('(local_field_940=bsbdigi20vr or local_field_940=bsbdigi20fs) and subjects=philosophie')
print(len(records), 'Ergebnisse')


Suchanfragen an die SRU-Schnittstelle werden in der Form "Index/Suchfeld=Suchbegriffe" formuliert, mit boole'schen Operatoren können Sie mehrere Suchfelder kombinieren. Eine Liste aller verfügbaren Suchindices finden Sie unter <https://bsb.alma.exlibrisgroup.com/view/sru/49BVB_BSB?operation=explain&version=1.2>. In unserem Beispiel suchen wir nach dem Schlagwort "Philosophie" in Kombination mit Digitalisaten aus dem [Digi20-Projekt](https://www.digitale-sammlungen.de/de/digi20/about). Der Klammerinhalt hinter bsb_sru kann leicht angepasst werden, um andere Suchanfragen auszuprobieren. Weitere Hilfe zur Formulierung von Suchen an die SRU-Schnittstelle und eine Beschreibung interessanter Indices finden Sie hier: 

SRU-Suchanfragen allgemein <https://www.bsb-muenchen.de/bsblab/datenschnittstellen/bsb-sru/#c24304>

Suchindices und mehr <https://www.bsb-muenchen.de/bsblab/werkzeugkasten/#c24521>

Als nächstes wird die Zahl der Treffer angezeigt.

## Ausgabe der Daten <a class="anchor" id="Teil4"></a>

Mit "Pandas" werden die Ergebnisse aus dem Dictionary als Dataframe ausgegeben. Dann werden die ersten und die letzten 5 Zeilen als "Sample" angezeigt und alle Ergebnisse mit "df.to_csv" in eine CSV-Datei geschrieben.

In [ ]:
output = [parse_record(record_data) for record_data in records]
df = pd.DataFrame(output)

# Display the first 5 lines
print("Sample View:")
print(df.head())
print(df.tail())

df.to_csv("Titelliste.csv", index=False)